In [1]:
import rasterio as rio
import matplotlib.pyplot as plt 
import numpy as np
from scipy import interpolate
import statsmodels.api as sm
import scipy.stats as st
import os, sys, pickle, gzip
import datetime
import geopy.distance
import xarray as xr
import cartopy.crs as ccrs


In [2]:
run ../util/setupConsole

In [3]:
dataDirDiscovery = '/dartfs-hpc/rc/lab/C/CMIG/ecoffel/data/projects/ag-land-climate'

heatwave_pct = 90

crop = 'Maize'
wxData = 'era5'

years = [1979, 2019]

In [4]:
sacksMaizeNc = xr.open_dataset('%s/sacks/Maize.crop.calendar.fill.nc'%dataDirDiscovery)
sacksStart = sacksMaizeNc['plant'].values
sacksStart = np.roll(sacksStart, -int(sacksStart.shape[1]/2), axis=1)
sacksStart[sacksStart < 0] = np.nan
sacksEnd = sacksMaizeNc['harvest'].values
sacksEnd = np.roll(sacksEnd, -int(sacksEnd.shape[1]/2), axis=1)
sacksEnd[sacksEnd < 0] = np.nan

sacksLat = np.linspace(90, -90, 360)
sacksLon = np.linspace(0, 360, 720)

In [5]:
heatwave_days = [] 

In [6]:
dsMax = xr.open_mfdataset('/dartfs-hpc/rc/lab/C/CMIG/ERA5/daily/tasmax_*.nc', decode_cf=False, concat_dim='time')
dims = dsMax.dims

tDt = []
i = 0
for curTTime in dsMax.time:
    if curTTime == 0:
        startingDate = datetime.datetime(years[0]+i, 1, 1, 0, 0, 0)
        i += 1
    delta = datetime.timedelta(days=int(curTTime.values))
    tDt.append(startingDate + delta)



In [7]:
dsMax['time'] = tDt
dsMax['mx2t'] = dsMax['mx2t'] - 273.15

In [8]:
dsMax_quantile = dsMax.chunk({'latitude':50, 'longitude':50, 'time': -1}).groupby('time.year').quantile(q=0.9, dim='time')

In [9]:
dsMax_quantile_midwest = dsMax_quantile.sel(latitude=slice(45,40), longitude=slice(250,260))

In [10]:
dsMax_quantile_midwest.load()

<xarray.Dataset>
Dimensions:    (latitude: 21, longitude: 41, year: 41)
Coordinates:
    quantile   float64 0.9
  * longitude  (longitude) float32 250.0 250.25 250.5 ... 259.5 259.75 260.0
  * latitude   (latitude) float32 45.0 44.75 44.5 44.25 ... 40.5 40.25 40.0
  * year       (year) int64 1979 1980 1981 1982 1983 ... 2016 2017 2018 2019
Data variables:
    mx2t       (year, latitude, longitude) float64 19.01 18.6 ... 32.05 32.23

In [ ]:
dsMax_quantile.load()

In [12]:
dsMax.time

<xarray.DataArray 'time' (time: 14975)>
array(['1979-01-01T00:00:00.000000000', '1979-01-02T00:00:00.000000000',
       '1979-01-03T00:00:00.000000000', ..., '1979-12-29T00:00:00.000000000',
       '1979-12-30T00:00:00.000000000', '1979-12-31T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-01-02 ... 1979-12-31

In [ ]:
for xlat in range(len(lat)):

    if xlat % 25 == 0:
        print('%.0f %% complete'%(xlat/len(lat)*100))

    for ylon in range(len(lon)):

        sacksNearestX = np.where((abs(sacksLat-lat[xlat]) == np.nanmin(abs(sacksLat-lat[xlat]))))[0][0]
        sacksNearestY = np.where((abs(sacksLon-lon[ylon]) == np.nanmin(abs(sacksLon-lon[ylon]))))[0][0]

        growingSeasonLen = 0

        if ~np.isnan(sacksStart[sacksNearestX,sacksNearestY]) and ~np.isnan(sacksEnd[sacksNearestX,sacksNearestY]):

            # in southern hemisphere when planting happens in fall and harvest happens in spring
            if sacksStart[sacksNearestX,sacksNearestY] > sacksEnd[sacksNearestX,sacksNearestY]:
                curTmax = xr.concat([tmaxLastYear[int(sacksStart[sacksNearestX,sacksNearestY]):, xlat, ylon], \
                                     tmax[:int(sacksEnd[sacksNearestX,sacksNearestY]), xlat, ylon]], dim='time')

                curTmin = xr.concat([tminLastYear[int(sacksStart[sacksNearestX,sacksNearestY]):, xlat, ylon], \
                                     tmin[:int(sacksEnd[sacksNearestX,sacksNearestY]), xlat, ylon]], dim='time')

                growingSeasonLen = (365-int(sacksStart[sacksNearestX,sacksNearestY])) + int(sacksEnd[sacksNearestX,sacksNearestY])

            else:
                curTmax = tmax[int(sacksStart[sacksNearestX,sacksNearestY]):int(sacksEnd[sacksNearestX,sacksNearestY]), xlat, ylon]
                curTmin = tmin[int(sacksStart[sacksNearestX,sacksNearestY]):int(sacksEnd[sacksNearestX,sacksNearestY]), xlat, ylon]

                growingSeasonLen = int(sacksEnd[sacksNearestX,sacksNearestY]) - int(sacksStart[sacksNearestX,sacksNearestY])

            # calc seasonal gdd/kdd
            curYearGdd = (curTmax.where(curTmax > t_low) + curTmin.where(curTmin > t_low))/2-t_low
            curYearKdd = curTmax.where(curTmax > t_high)-t_high

            # loop over weeks to get weekly kdd/gdd

            for w, wInd in enumerate(range(0, growingSeasonLen, 7)):
                gddWeekly[xlat, ylon, w] = np.nansum(curYearGdd.values[wInd:wInd+7])
                kddWeekly[xlat, ylon, w] = np.nansum(curYearKdd.values[wInd:wInd+7])

            curYearGdd = curYearGdd.sum(dim='time')
            gdd[xlat, ylon] = curYearGdd.values

            curYearKdd = curYearKdd.sum(dim='time')
            kdd[xlat, ylon] = curYearKdd.values